In [ ]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [4]:
retriever= WikipediaRetriever(top_k_results=2, lang="en")

In [5]:
topics = [
    "India–Pakistan relations",
    "China–Pakistan relations",
    "Sino-Indian relations"
]

docs = []
for t in topics:
    docs.extend(retriever.invoke(t))

print(len(docs))


6


In [6]:
docs

[Document(metadata={'title': 'India–Pakistan relations', 'summary': 'India and Pakistan have a complex and largely hostile relationship that is rooted in a multitude of historical and political events, most notably the partition of British India in August 1947.\nTwo years after World War II, the United Kingdom formally dissolved British India, dividing it into two new sovereign nations: the Union of India and Pakistan. The partitioning of the former British colony resulted in the displacement of up to 15 million people, with the death toll estimated to have reached between several hundred thousand and one million people as Hindus and Muslims migrated in opposite directions across the Radcliffe Line to reach India and Pakistan, respectively. In 1950, India emerged as a secular republic with a Hindu-majority population. Shortly afterwards, in 1956, Pakistan emerged as an Islamic republic with a Muslim-majority population.\nWhile the two South Asian countries established full diplomatic t

## Vector Store Retriever

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

/Users/virenderparasariya/Desktop/Projects/langchain/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
documents = [

    Document(
        page_content=(
            "A skilled thief who steals corporate secrets by entering dreams is "
            "offered a chance to erase his criminal past by performing inception, "
            "the act of planting an idea into a target’s subconscious. The mission "
            "spans multiple dream layers, blurring the boundary between reality and illusion."
        ),
        metadata={
            "title": "Inception",
            "year": 2010,
            "genre": ["Sci-Fi", "Thriller"],
            "themes": ["dreams", "identity", "time", "reality"],
            "director": "Christopher Nolan"
        }
    ),

    Document(
        page_content=(
            "Batman confronts the Joker, a criminal mastermind who aims to destabilize "
            "Gotham City by exposing the fragility of moral and social order. The conflict "
            "forces Batman to question the ethical limits of vigilantism."
        ),
        metadata={
            "title": "The Dark Knight",
            "year": 2008,
            "genre": ["Action", "Crime", "Drama"],
            "themes": ["chaos", "morality", "heroism"],
            "director": "Christopher Nolan"
        }
    ),

    Document(
        page_content=(
            "In a future where Earth is becoming uninhabitable, astronauts travel "
            "through a wormhole in search of a new planet for humanity. The journey "
            "explores time dilation, sacrifice, and the enduring power of human love."
        ),
        metadata={
            "title": "Interstellar",
            "year": 2014,
            "genre": ["Sci-Fi", "Drama"],
            "themes": ["space", "time", "survival", "love"],
            "director": "Christopher Nolan"
        }
    ),

    Document(
        page_content=(
            "A hacker discovers that reality is a simulated world created by machines "
            "to enslave humanity. Joining a rebel group, he learns to bend the rules "
            "of the simulation while questioning free will and identity."
        ),
        metadata={
            "title": "The Matrix",
            "year": 1999,
            "genre": ["Sci-Fi", "Action"],
            "themes": ["simulation", "free will", "identity"],
            "director": "The Wachowskis"
        }
    ),

    Document(
        page_content=(
            "A kind-hearted man with limited intelligence unintentionally influences "
            "major historical events in the United States while pursuing love and "
            "friendship across several decades of social change."
        ),
        metadata={
            "title": "Forrest Gump",
            "year": 1994,
            "genre": ["Drama", "Romance"],
            "themes": ["love", "destiny", "history"],
            "director": "Robert Zemeckis"
        }
    ),

    Document(
        page_content=(
            "A poor family infiltrates a wealthy household by posing as unrelated "
            "professionals. As their deception deepens, hidden tensions erupt, "
            "revealing sharp critiques of class inequality and social hierarchy."
        ),
        metadata={
            "title": "Parasite",
            "year": 2019,
            "genre": ["Thriller", "Drama"],
            "themes": ["class conflict", "inequality", "deception"],
            "director": "Bong Joon-ho"
        }
    ),

    Document(
        page_content=(
            "A betrayed Roman general is reduced to slavery and forced into gladiatorial combat. "
            "Rising through the arena, he seeks justice and revenge against the corrupt emperor "
            "who destroyed his family."
        ),
        metadata={
            "title": "Gladiator",
            "year": 2000,
            "genre": ["Action", "Drama", "Historical"],
            "themes": ["revenge", "honor", "power"],
            "director": "Ridley Scott"
        }
    ),

    Document(
        page_content=(
            "An insomniac office worker forms an underground fight club with a charismatic "
            "anarchist, which evolves into a radical movement challenging consumer culture "
            "and personal identity."
        ),
        metadata={
            "title": "Fight Club",
            "year": 1999,
            "genre": ["Drama", "Psychological"],
            "themes": ["identity", "consumerism", "anarchy"],
            "director": "David Fincher"
        }
    ),

    Document(
        page_content=(
            "Wrongly imprisoned for murder, a quiet banker forms deep friendships "
            "inside a brutal prison system while secretly planning a path to freedom, "
            "demonstrating the enduring power of hope and perseverance."
        ),
        metadata={
            "title": "The Shawshank Redemption",
            "year": 1994,
            "genre": ["Drama"],
            "themes": ["hope", "friendship", "freedom"],
            "director": "Frank Darabont"
        }
    ),

    Document(
        page_content=(
            "Multiple crime stories intersect in a non-linear narrative involving "
            "gangsters, boxers, and hitmen in Los Angeles. The film blends dark humor "
            "with violence while exploring fate, chance, and redemption."
        ),
        metadata={
            "title": "Pulp Fiction",
            "year": 1994,
            "genre": ["Crime", "Drama"],
            "themes": ["fate", "violence", "redemption"],
            "director": "Quentin Tarantino"
        }
    ),
]


In [9]:
def normalize_metadata(metadata: dict) -> dict:
    normalized = {}
    for k, v in metadata.items():
        if isinstance(v, list):
            normalized[k] = ", ".join(map(str, v))
        else:
            normalized[k] = v
    return normalized


documents = [
    Document(
        page_content=d.page_content,
        metadata=normalize_metadata(d.metadata)
    )
    for d in documents
]


In [10]:
documents

[Document(metadata={'title': 'Inception', 'year': 2010, 'genre': 'Sci-Fi, Thriller', 'themes': 'dreams, identity, time, reality', 'director': 'Christopher Nolan'}, page_content='A skilled thief who steals corporate secrets by entering dreams is offered a chance to erase his criminal past by performing inception, the act of planting an idea into a target’s subconscious. The mission spans multiple dream layers, blurring the boundary between reality and illusion.'),
 Document(metadata={'title': 'The Dark Knight', 'year': 2008, 'genre': 'Action, Crime, Drama', 'themes': 'chaos, morality, heroism', 'director': 'Christopher Nolan'}, page_content='Batman confronts the Joker, a criminal mastermind who aims to destabilize Gotham City by exposing the fragility of moral and social order. The conflict forces Batman to question the ethical limits of vigilantism.'),
 Document(metadata={'title': 'Interstellar', 'year': 2014, 'genre': 'Sci-Fi, Drama', 'themes': 'space, time, survival, love', 'director

In [11]:
embedding_model=OpenAIEmbeddings()

vectorstore =Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="movie-plots"
)

In [12]:
retriever2=vectorstore.as_retriever(search_kwargs={"k":2})

In [13]:
query="Which movie do you think is inspirational?"
results=retriever2.invoke(query)

In [14]:
results

[Document(metadata={'genre': 'Drama', 'director': 'Frank Darabont', 'year': 1994, 'themes': 'hope, friendship, freedom', 'title': 'The Shawshank Redemption'}, page_content='Wrongly imprisoned for murder, a quiet banker forms deep friendships inside a brutal prison system while secretly planning a path to freedom, demonstrating the enduring power of hope and perseverance.'),
 Document(metadata={'title': 'Forrest Gump', 'genre': 'Drama, Romance', 'year': 1994, 'themes': 'love, destiny, history', 'director': 'Robert Zemeckis'}, page_content='A kind-hearted man with limited intelligence unintentionally influences major historical events in the United States while pursuing love and friendship across several decades of social change.')]

In [15]:
for i,doc in enumerate(results):
    print(f"\n-- Result {i+1}--")
    print(doc.page_content)


-- Result 1--
Wrongly imprisoned for murder, a quiet banker forms deep friendships inside a brutal prison system while secretly planning a path to freedom, demonstrating the enduring power of hope and perseverance.

-- Result 2--
A kind-hearted man with limited intelligence unintentionally influences major historical events in the United States while pursuing love and friendship across several decades of social change.


#MMR Retriever

In regular similarity search, you may get documents that are:
All very similar to each other
Repeating the same info
Lacking diverse perspectives


MMR Retriever avoids that by: 
Picking the most relevant document first
Them picking the next most relevant and least similar to already selected dodcs
And so on..

Useful when documents have semantic overlap

In [3]:
documents = [
    Document(
        page_content="Climate change is causing rising global temperatures and an increase in extreme weather events across the world."
    ),

    Document(
        page_content="Global warming has resulted in more frequent heatwaves, floods, and powerful storms worldwide."
    ),

    Document(
        page_content="Human activities such as burning fossil fuels and deforestation are accelerating climate change."
    ),

    Document(
        page_content="Increasing concentrations of greenhouse gases are leading to more extreme and unpredictable weather patterns."
    ),

    Document(
        page_content="Renewable energy sources like solar and wind power can help reduce carbon emissions and slow global warming."
    ),

    Document(
        page_content="The continued consumption of fossil fuels is a major contributor to global greenhouse gas emissions.",
    ),

    Document(
        page_content="Solar and wind energy are important alternatives to coal and oil for achieving sustainable development."
    ),

    Document(
        page_content="Reducing carbon emissions is essential to limit the long-term impacts of climate change."
    ),

    Document(
        page_content="Government policies and environmental regulations play a critical role in combating climate change."
    ),

    Document(
        page_content="International cooperation is necessary to effectively address global climate and environmental challenges."
    ),
]


In [4]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.vectorstores import FAISS

embedding_model= OpenAIEmbeddings()

vectorstore=FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

In [6]:
retriever3 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "fetch_k": 12,
        "lambda_mult": 0.5
    }
)


In [ ]:
query="What causes climate change?"
results=retriever3.invoke(query)

In [5]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip uninstall langchain -y
%pip install "langchain<1.0"

Found existing installation: langchain 1.2.7
Uninstalling langchain-1.2.7:
  Successfully uninstalled langchain-1.2.7
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s  0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
  Attempting uninstall: langchain-text-splitters 0/3 [langchain-core]
    Found existing installation: langchain-text-splitters 1.1.0langchain-core]
    Uninstalling langchain-text-splitters-1.1.0: 0/3 [langchain-core]
      Successfully uninstalled langchain-text-splitters-1.1.0 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain]/3 [langchain]text-splitters]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflic

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever


In [7]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [8]:
# Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [9]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [10]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)

In [11]:
# Query
query = "How to improve energy levels and maintain balance?"

In [12]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

: 

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)